In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

bytedance


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

doc_test_x.npy		  lr_ct.csv
doc_train_x.npy		  lr_doc2vec_last500w.csv
doc_train_y.npy		  lr_doc2vec_one500w.csv
fasttext.csv		  model11.bin
fasttext_fist1000W.csv	  model.bin
fasttext_lrct.csv	  test_feature.npy
kesci_submit		  test.txt
labeled_content		  train.txt
lgb_doc2vec_last500w.csv  valid.txt
linear_tfidf.csv	  x_train_feature_last1000W.npy
lost+found		  x_train_test_feature_last1000W.npy
lr_ct_1000wEoch.csv	  y_train_feature_last1000W.npy
lr_ct_100万.csv		  y_train_test_feature_last1000W.npy
lr_ct_500wEach.csv


In [3]:
# 查看当前kernerl下的package
# !pip list --format=columns

In [4]:
# 显示cell运行时长
%load_ext klab-autotime

In [5]:
import numpy as np
import pandas as pd
import time
import math
from sklearn.model_selection import KFold
from fastText import train_supervised
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
import lightgbm as lgb
import Levenshtein
import gc

time: 1.14 s


In [6]:
path="/home/kesci/input/bytedance/first-round/"
train = pd.read_csv(path+'train.csv', header=None, chunksize=10000000)
test = pd.read_csv(path+'test.csv', header=None)
pre_df = test[[0, 2]]

time: 9.93 s


In [7]:
def lcsubstr_lens(s1, s2): 
    m=[[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]  #生成0矩阵，为方便后续计算，比字符串长度多了一列
    mmax=0   #最长匹配的长度
    p=0  #最长匹配对应在s1中的最后一位
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i]==s2[j]:
                m[i+1][j+1]=m[i][j]+1
                if m[i+1][j+1]>mmax:
                    mmax=m[i+1][j+1]
                    p=i+1
    return mmax


#
def lcseque_lens(s1, s2): 
     # 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
    m = [ [ 0 for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    # d用来记录转移方向
    d = [ [ None for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 

    for p1 in range(len(s1)): 
        for p2 in range(len(s2)): 
            if s1[p1] == s2[p2]:            #字符匹配成功，则该位置的值为左上方的值加1
                m[p1+1][p2+1] = m[p1][p2]+1
                d[p1+1][p2+1] = 'ok'          
            elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向
                m[p1+1][p2+1] = m[p1+1][p2] 
                d[p1+1][p2+1] = 'left'          
            else:                           #上值大于左值，则该位置的值为上值，并标记方向up
                m[p1+1][p2+1] = m[p1][p2+1]   
                d[p1+1][p2+1] = 'up'         
    (p1, p2) = (len(s1), len(s2)) 
    s = [] 
    while m[p1][p2]:    #不为None时
        c = d[p1][p2]
        if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个
            s.append(s1[p1-1])
            p1 -= 1
            p2 -= 1 
        if c == 'left':  #根据标记，向左找下一个
            p2 -= 1
        if c == 'up':   #根据标记，向上找下一个
            p1 -= 1
    return len(s)
    
def compute_cosine(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算余弦相似度
    sum = 0
    sq1 = 0
    sq2 = 0
    for i in range(len(vect1)):
        sum += vect1[i] * vect2[i]
        sq1 += pow(vect1[i], 2)
        sq2 += pow(vect2[i], 2)
    try:
        result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
    except ZeroDivisionError:
        result = 0.0
    return result

def Pehrson(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算Pehrson
    x = np.vstack([vect1, vect2])
    return np.corrcoef(x)[0][1]

time: 23.8 ms


In [8]:
def getfeature(df):
    # df.drop_duplicates(inplace=True)
    since = time.time()
    
    df['query_len'] = df.apply(lambda x: len(x[1].split(' ')), axis=1)
    df['query_len'] = df['query_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('1complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def get_query_unique(x):
        a = len(set(x[1].split(' '))-set(x[3].split(' ')))
        return a
    df['query_unique'] = df.apply(get_query_unique, axis=1)
    df['query_unique'] = df['query_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#####################################3
    print('2complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['title_len'] = df.apply(lambda x: len(x[3].split(' ')), axis=1)
    df['title_len'] = df['title_len'].astype(np.int32)
    
    time_elapsed = time.time() - since######################################
    print('3complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def get_query_unique(x):
        a = len(set(x[3].split(' '))-set(x[1].split(' ')))
        return a
    
    df['title_unique'] = df.apply(get_query_unique, axis=1)
    df['title_unique'] = df['title_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['dif'] = (df['query_len'] - df['title_len']).abs()
    df['dif'] = df['dif'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('4complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def query_title_same_len(x):
        a = len(x[3].split(' ') and x[1].split(' '))
        return a
    df['query_title_same_len'] = df.apply(query_title_same_len, axis=1)
    df['query_title_same_len'] = df['query_title_same_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('5complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    def samelen_query_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[1].split(' '))
    df['samelen_query_rate'] = df.apply(samelen_query_rate, axis=1)
    df['samelen_query_rate'] = df['samelen_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('6complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def samelen_title_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[3].split(' '))
    df['samelen_title_rate'] = df.apply(samelen_title_rate, axis=1)
    df['samelen_title_rate'] = df['samelen_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('7complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    
    def q_t_all_len(x):
        a = len(x[1].split(' ')) + len(x[3].split(' '))
        return a
    df['q_t_all_len'] = df.apply(q_t_all_len, axis=1)
    df['q_t_all_len'] = df['q_t_all_len'].astype(np.int32)
    
    time_elapsed = time.time() - since##########################################
    print('8complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_all_word_len(x):
        a = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return a
    df['q_t_all_word_len'] = df.apply(q_t_all_word_len, axis=1)
    df['q_t_all_word_len'] = df['q_t_all_word_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('9complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['q_t_diff_len'] = df.apply(lambda x: len(set(x[1].split(' '))^set(x[3].split(' '))), axis=1)
    df['q_t_diff_len'] = df['q_t_diff_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('9complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_diff_q_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[1].split(' ')))
    df['q_t_diff_q_rate'] = df.apply(q_t_diff_q_rate, axis=1)
    df['q_t_diff_q_rate'] = df['q_t_diff_q_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('10complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def q_t_diff_t_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[3].split(' ')))
    df['q_t_diff_t_rate'] = df.apply(q_t_diff_t_rate, axis=1)
    df['q_t_diff_t_rate'] = df['q_t_diff_t_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('11complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    
    def query_diff_title(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['query_diff_title'] = df.apply(query_diff_title, axis=1)
    df['query_diff_title'] = df['query_diff_title'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('12complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def query_diff_title_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['query_diff_title_rate'] = df.apply(query_diff_title_rate, axis=1)
    df['query_diff_title_rate'] = df['query_diff_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('13complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def title_diff_query(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['title_diff_query'] = df.apply(title_diff_query, axis=1)
    df['title_diff_query'] = df['title_diff_query'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('14complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def title_diff_query_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['title_diff_query_rate'] = df.apply(title_diff_query_rate, axis=1)
    df['title_diff_query_rate'] = df['title_diff_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since
    print('15complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def dice(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' '))) + len(set(x[3].split(' ')))
        return np.float32(2*a1/a2)
    df['dice'] = df.apply(dice, axis=1)
    df['dice'] = df['dice'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('16complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    def jaccord(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return np.float32(a1/a2)
    df['jaccord'] = df.apply(jaccord, axis=1)
    df['jaccord'] = df['jaccord'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('17complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['query_count'] = df.groupby(1)[1].transform('count')
    df['query_count'] = df['query_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('18complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['title_count'] = df.groupby(3)[3].transform('count')
    df['title_count'] = df['title_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('19complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['lcseque_lens'] = df.apply(lambda x: lcseque_lens(x[1], x[3]), axis=1)
    df['lcseque_lens'] = df['lcseque_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('20complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['lcsubstr_lens'] = df.apply(lambda x: lcsubstr_lens(x[1], x[3]), axis=1)
    df['lcsubstr_lens'] = df['lcsubstr_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('21complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    

    df['ratio'] = df.apply(lambda x: Levenshtein.ratio(x[1], x[3]), axis=1)
    df['ratio'] = df['ratio'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('22complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    

    df['jaro'] = df.apply(lambda x: Levenshtein.jaro(x[1], x[3]), axis=1)
    df['jaro'] = df['jaro'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('23complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['jaro_winkler'] = df.apply(lambda x: Levenshtein.jaro_winkler(x[1], x[3]), axis=1)
    df['jaro_winkler'] = df['jaro_winkler'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('24complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    df['cosine_similarity'] = df.apply(lambda x: compute_cosine(x[1], x[3]), axis=1)
    df['cosine_similarity'] = df['cosine_similarity'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('25complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()

    df['levenshtein'] = df.apply(lambda x: Levenshtein.distance(x[1], x[3]), axis=1)
    df['levenshtein'] = df['levenshtein'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('26complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    
    df['Pehrson'] = df.apply(lambda x:Pehrson(x[1], x[3]), axis=1)
    df['Pehrson'] = df['Pehrson'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('27complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    since = time.time()
    
    df['list_dis'] = df.apply(lambda x: Levenshtein.seqratio(x[1].split(' '), x[3].split(' ')), axis=1)
    df['list_dis'] = df['list_dis'].astype(np.float32)
    
    time_elapsed = time.time() - since
    print('28complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    

    try:
        df.drop([0, 1, 2, 3, 4], axis=1, inplace=True)
    except:
        df.drop([0, 1, 2, 3], axis=1, inplace=True)
        
    return df

time: 25.3 ms


In [28]:
x = 1
for df in train:
    if x != 10:
        x += 1
        continue
    
    print('第%d次处理开始'%x)
    
    print('训练集测试集合并')
    y_train = df[4]
    df.drop([4], axis=1, inplace=True)
    len_train = len(df)
    df = pd.concat((df, test))
    del test
    print('合并完成，提取特征')
    
    df = getfeature(df)
    df['Pehrson'] = df['Pehrson'].fillna(df['Pehrson'].mean())
    df = df.values
    y_train = y_train.values
    np.save('x_train_test_feature_last1000W.npy', df)
    np.save('y_train_test_feature_last1000W.npy', y_train)

    print('第%d次处理完成，开始训练'%x)
    break

第10次处理开始
训练集测试集合并
合并完成，提取特征
1complete in 5m 4s
2complete in 8m 14s
3complete in 4m 49s
complete in 8m 28s
4complete in 0m 0s
5complete in 7m 15s
6complete in 10m 58s
7complete in 10m 46s
8complete in 7m 13s
9complete in 8m 17s
9complete in 8m 38s
10complete in 11m 25s
11complete in 11m 10s


In [29]:
print('划分数据集')
x_train = df[:len_train]
x_test = df[len_train:]
del df
print(x_train.shape, x_test.shape)

In [41]:
kf_n = 5
kf = KFold(n_splits=kf_n, shuffle=True)

pre_lr = 0

lr = LogisticRegression(penalty='l1', C=1, n_jobs=4)

for x_tr, x_vail in kf.split(x_train):
        
        lr.fit(x_train[x_tr], y_train[x_tr])
        print(metrics.roc_auc_score(y_train[x_vail], lr.predict_proba(x_train[x_vail])[:, 1]))
        print(metrics.log_loss(y_train[x_vail], lr.predict_proba(x_train[x_vail])[:, 1]))
        pre_lr += lr.predict_proba(x_test)[:, 1]

pre_lr /= kf_n

pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
pre_df['prediction'] = pre_lr
pre_df.to_csv('lr_doc2vec_last500w.csv', index=False, header=False)

In [ ]:
pre_lgb = 0

random_state = 2

# lgb_params = {
#     "objective" : "binary",
#     # "metric" : "auc",
#     "boosting": 'gbdt',
#     "max_depth" : -1,
#     "num_leaves" : 5,
#     "learning_rate" : 0.1,
#     "bagging_freq": 5,
#     "bagging_fraction" : 0.6,
#     "feature_fraction" : 0.05,
#     "min_data_in_leaf": 700,
#     "min_sum_heassian_in_leaf": 10,
#     "tree_learner": "serial",
#     "boost_from_average": "false",
# #    "lambda_l1" : 5,
# #    "lambda_l2" : 5,
#     "bagging_seed" : random_state,
#     "verbosity" : 1,
#     "seed": random_state
# }

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': "binary",
    'metric': ['binary_logloss',"auc"],
    'num_leaves': 10,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1,
    "seed": random_state,
    "bagging_seed" : random_state
}


for x_tr, x_vail in kf.split(x_train):
        
        #当x_train为ndarray时，下列数据集的划分会进行复制而占用内存
        
        # x_t1 = x_train[x_tr]
        # y_t1 = y_train[x_tr]
        
        # x_t2 = x_train[x_vail]
        # y_t2 = y_train[x_vail]
            
        lgbtr1=lgb.Dataset(x_train[x_tr], y_train[x_tr])
        lgbtr2=lgb.Dataset(x_train[x_vail], y_train[x_vail])
            
        evals_result = {}
        gbm = lgb.train(lgb_params,
                          lgbtr1,
                          num_boost_round=1000,
                          valid_sets=lgbtr2,
                          verbose_eval=50,
                          early_stopping_rounds=100,
                          evals_result=evals_result)
                          
        print(metrics.roc_auc_score(y_train[x_vail], gbm.predict(x_train[x_vail], num_iteration=gbm.best_iteration)))
        pre_lgb += gbm.predict(x_test, num_iteration=gbm.best_iteration)
        
pre_lgb /= kr_n

pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
pre_df['prediction'] = pre_lgb
pre_df.to_csv('lgb_doc2vec_last500w.csv', index=False, header=False)
